This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import json
import requests

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-01-02&api_key={}'.format(API_KEY)
r = requests.get(url)
afx_x_data = r.json()

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
    
#[print(key + ':', value) for key, value in afx_x_data.items()]

print(json.dumps(afx_x_data, indent=4))

{
    "dataset_data": {
        "limit": null,
        "transform": null,
        "column_index": null,
        "column_names": [
            "Date",
            "Open",
            "High",
            "Low",
            "Close",
            "Change",
            "Traded Volume",
            "Turnover",
            "Last Price of the Day",
            "Daily Traded Units",
            "Daily Turnover"
        ],
        "start_date": "2017-01-01",
        "end_date": "2017-01-02",
        "frequency": "daily",
        "data": [
            [
                "2017-01-02",
                34.99,
                35.94,
                34.99,
                35.8,
                null,
                44700.0,
                1590561.0,
                null,
                null,
                null
            ]
        ],
        "collapse": null,
        "order": null
    }
}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
# data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017
afx2017_url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&collapse=daily&api_key={}'.format(API_KEY)

r = requests.get(afx2017_url)
afx_dict = r.json() # returns python dictionary called afx_dict

#print(json.dumps(afx_dict, indent=4))

In [6]:
## what the highest and lowest opening prices were for the stock in this period?
opens = [x[1] for x in afx_dict["dataset_data"]["data"] if x[1]!=None]
high, low = max(opens), min(opens)
print("${} was the highest opening price in 2017.".format(high)) # highest opening price in 2017
print("${} was the lowest opening price in 2017.".format(low)) # lowest opening price in 2017

$53.11 was the highest opening price in 2017.
$34.0 was the lowest opening price in 2017.


In [7]:
## What was the largest change in any one day (based on High and Low price)?
changes = [x[2]-x[3] for x in afx_dict["dataset_data"]["data"] if x[2]!=None and x[3]!=None]

print("${} was the largest change in any one day based on the High and Low prices.".format(max(changes)))

$2.8100000000000023 was the largest change in any one day based on the High and Low prices.


In [8]:
## What was the largest change between any two days (based on Closing Price)?
def closing_diff(list_of_lists): #takes a list of lists
    closings = []
    diffs = []
    for list in list_of_lists:
        closings.append(list[4])
    for i in range(1, len(closings)):
        if closings[i] == None:
            continue
        elif closings[i] != None:
            diffs.append(abs(closings[i] - closings[i-1]))
    print("${} is the largest change, based on Closing Price, that occurred between any two days in 2017.".format(max(diffs)))
    
LOL = afx_dict["dataset_data"]["data"]
closing_diff(LOL)

$2.559999999999995 is the largest change, based on Closing Price, that occurred between any two days in 2017.


In [9]:
## What was the average daily trading volume during this year?
vols = [x[6] for x in afx_dict["dataset_data"]["data"] if x[6]!=None]
avg = (sum(vols)/len(vols))
print("The average daily trading volume during 2017 was {}.".format(avg))

The average daily trading volume during 2017 was 89124.33725490196.


In [10]:
## What was the median trading volume during this year?
def median(list):
    if len(list) == 0 or len(list) == 1:
        return "try again"
    elif len(list) % 2 == 0:
        h = int(len(list) / 2)
        return (list[h]+list[h+1]) / 2
    else: #if it's odd
        x = int((len(list)) / 2)
        return list[x]
    
print("The median trading volume during 2017 was {}.".format(median(vols)))

The median trading volume during 2017 was 134965.0.
